<a href="https://colab.research.google.com/github/mohith-vishwanath/Image_Classification_Kaggle_Cats_vs_Dogs/blob/main/PyTorch_Cats_vs_Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PyTorch Implementation

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

from torch.utils.data import DataLoader

import numpy as np

import itertools
import os, shutil, time
import random, glob
import matplotlib.pyplot as plt
import cv2

from google.colab import drive
drive.mount('/content/drive')

In [3]:
device = ""
if torch.cuda.is_available():
    device="cuda:0"
    print("GPU available")
else:
    device="cpu"
    print("GPU not available")

GPU available


In [8]:
IMG_SIZE = 224
CATEGORIES = ["cat","dog"]
BATCH_SIZE = 100
VAL_SPLIT = 0.2
EPOCHS = 5

In [4]:
def get_total_correct_predictions(preds,targets):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [6]:
def load_images_from_root(root,batch_size):
    transformations = transforms.Compose([
        transforms.Grayscale(1),
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        transforms.ToTensor()
    ])

    train_data = ImageFolder(root=root,transform=transformations)
    train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)

    print("Class Encoding")
    print(train_data.class_to_idx)

    return train_data, train_loader

In [9]:
train_images_path = "/content/drive/MyDrive/Datasets/Cats_vs_Dogs/training/"
train_data, train_loader = load_images_from_root(train_images_path,100)

Class Encoding
{'cat': 0, 'dog': 1}


In [10]:
pytorch_model = nn.Sequential(
    nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2,stride=2),
    nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2,stride=2),
    nn.Flatten(start_dim=1),
    nn.Linear(in_features=12*53*53,out_features=120),
    nn.ReLU(),
    nn.Linear(in_features=120,out_features=40),
    nn.ReLU(),
    nn.Linear(in_features=40,out_features=10),
    nn.ReLU(),
    nn.Linear(in_features=10,out_features=2),
)

In [11]:
pytorch_model = pytorch_model.to(device)
print(pytorch_model)

Sequential(
  (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=33708, out_features=120, bias=True)
  (8): ReLU()
  (9): Linear(in_features=120, out_features=40, bias=True)
  (10): ReLU()
  (11): Linear(in_features=40, out_features=10, bias=True)
  (12): ReLU()
  (13): Linear(in_features=10, out_features=2, bias=True)
)


In [ ]:
print(f'Total number of batches : {BATCH_SIZE}')
print(f'Running on : {device}')
print(f'Number of epochs : {EPOCHS}')
time.sleep(0.5)

optimizer = optim.Adam(pytorch_model.parameters(),lr=0.001)
for epoch in range(EPOCHS):
    total_correct = 0
    for batch in tqdm.tqdm(train_loader,desc=f'Epoch {epoch} '):
        optimizer.zero_grad()
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        prediction = pytorch_model(images)
        loss = F.cross_entropy(prediction,labels)
        loss.backward()
        optimizer.step()
        total_correct += get_total_correct_predictions(prediction,labels)
    print(f'Epoch {epoch} | Accuracy {round((total_correct/len(train_loader))*100,3)}%')

Total number of batches : 100
Running on : cuda:0
Number of epochs : 5




Epoch 0 :   0%|          | 0/251 [00:00<?, ?it/s]

Epoch 0 :   0%|          | 1/251 [00:23<1:36:39, 23.20s/it]

Epoch 0 :   1%|          | 2/251 [00:45<1:34:49, 22.85s/it]

Epoch 0 :   1%|          | 3/251 [01:06<1:32:39, 22.42s/it]

Epoch 0 :   2%|▏         | 4/251 [01:28<1:31:08, 22.14s/it]

Epoch 0 :   2%|▏         | 5/251 [01:49<1:30:00, 21.95s/it]

Epoch 0 :   2%|▏         | 6/251 [02:11<1:29:48, 21.99s/it]

Epoch 0 :   3%|▎         | 7/251 [02:33<1:28:42, 21.82s/it]

Epoch 0 :   3%|▎         | 8/251 [02:54<1:28:20, 21.81s/it]

Epoch 0 :   4%|▎         | 9/251 [03:17<1:28:16, 21.89s/it]

Epoch 0 :   4%|▍         | 10/251 [03:38<1:27:59, 21.91s/it]

Epoch 0 :   4%|▍         | 11/251 [03:59<1:26:29, 21.62s/it]

Epoch 0 :   5%|▍         | 12/251 [04:23<1:27:58, 22.09s/it]

Epoch 0 :   5%|▌         | 13/251 [04:44<1:27:03, 21.95s/it]

Epoch 0 :   6%|▌         | 14/251 [05:07<1:27:35, 22.17s/it]

Epoch 0 :   6%|▌         | 15/251 [05:30<1:27:59, 22.37s/it]

Epoch 0 :   6%|▋         |